<h1> Script predicción</h1> 
<h2> The Backpropagation Boyz </h2>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns

# Agrupar datos

In [ ]:
!sort Modelar_UH2022.txt > Modelar_UH2022_sorted.txt
!sed '1h;1d;$!H;$!d;G' Modelar_UH2022_sorted.txt > tmp.txt; mv tmp.txt Modelar_UH2022_sorted.txt
!rm Modelar_UH2022_agg.txt

In [ ]:
file1 = open('Modelar_UH2022_sorted.txt', 'r')
file2 = open('Modelar_UH2022_agg.txt', 'a')
iden_pre = ""
curr_delta = 0
curr_reading = 0

for i, line in enumerate(file1.readlines()):
    if i != 0:
        iden, content  = line.split()
        content = content.split('|')

        if '' in content: content = ['0' if c == '' else c for c in content]

        if iden != iden_pre and i!= 1:

            file2.write(iden_pre+'|'+ str(curr_reading)+'|'+str(curr_delta)+'\n')
            curr_delta = 0

            
        curr_reading = int(content[1])*100 + int(content[2])*(10**(2-len(content[2])))

        curr_delta += int(content[3])*100 + int(content[4])*(10**(2-len(content[2])))

        iden_pre = iden

    else:
        file2.write('ID|SAMPLETIME|READING|DELTA\n')

file1.close()
file2.close()

In [ ]:
!rm Modelar_UH2022_sorted.txt

# Foo

In [ ]:
ds = pd.read_csv("Modelar_UH2022_agg.txt", sep='|')
print(ds.head)
print(ds.shape)

In [ ]:
ds = ds.sort_values(by=['ID', 'SAMPLETIME'])

In [ ]:
ts = []

for serie in ds['ID'].unique():
    if serie% 20 == 0 :print('reading serie:'+ str(serie))
    
    serie_samples = pd.Series(data=np.zeros(365),index=ds['SAMPLETIME'].unique())
    sample = ds.loc[ds['ID'] == serie, 'DELTA']
    sample.index = ds.loc[ds['ID'] == serie, 'SAMPLETIME']
    serie_samples.loc[ds.loc[(ds['ID'] == serie), 'SAMPLETIME']] = sample
    # serie_samples = ds.loc[(ds['ID'] == serie) &  (ds['SAMPLETIME'] == ds['SAMPLETIME'].unique()), 'DELTA']
    # print(len(serie_samples))
    # for moment in ds['SAMPLETIME'].unique():
    #     sample = serie_samples = ds.loc[(ds['ID'] == serie) &  (ds['SAMPLETIME'] == moment), 'DELTA']
    #     serie_samples.append(sample)
    ts.append(serie_samples)
ts = np.array(ts)

In [ ]:
from sklearn.model_selection import train_test_split
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler
scaler = TimeSeriesScalerMeanVariance()
ts_norm = scaler.fit_transform(ts)

X_train, X_test, ind_train, ind_test = train_test_split(ts_norm, range(len(ts_norm)), test_size=0.8)


In [ ]:
km = TimeSeriesKMeans(n_clusters=10, metric="dtw")
y_train = km.fit_predict(X_train)

In [ ]:
y_test = km.predict(X_test)

In [ ]:
ind = np.concatenate([ind_train, ind_test])
ids2 = np.concatenate([ids_train, ids_test])
X = np.concatenate([X_train, X_test])
y = np.concatenate([y_train, y_test])

In [ ]:
for yi in range(n):
    print(sum(y == yi))